# 🏀 NBA API - V3 Endpoint Normalization Fix Demo

This notebook demonstrates the fix for [Issue #602](https://github.com/swar/nba_api/issues/602) - `get_normalized_json()` and `get_normalized_dict()` returning empty results for V3 endpoints.

## The Problem

V3 endpoints like `BoxScoreTraditionalV3` were returning:
- ✅ `get_dict()` - Works fine, returns raw nested JSON
- ❌ `get_normalized_json()` - Returns empty `{}`
- ❌ `get_normalized_dict()` - Returns empty `{}`

This made it difficult to work with V3 endpoints in a consistent way.

## The Solution

The fix adds V3 endpoint support to the normalization methods by:
1. Tracking which endpoint generated the response
2. Using endpoint-specific parsers to normalize the data
3. Converting nested JSON to flat, tabular format

---

## 📦 Installation

### Option 1: Install the fix from the PR branch (recommended for testing)

In [ ]:
# Install from the fix branch
!pip install -q git+https://github.com/brandonhawi/nba_api.git@fix/issue-602-normalized-methods

---

## 🔍 Demo: Before vs After

Let's fetch box score data from a recent game and compare the outputs.

In [ ]:
from nba_api.stats.endpoints import boxscoretraditionalv3
import json

# Fetch box score for a recent game (Timberwolves vs Suns, 11/22/2024)
response = boxscoretraditionalv3.BoxScoreTraditionalV3(game_id="0022500053")

print("📊 BoxScoreTraditionalV3 endpoint initialized")
print(f"Game ID: 0022500053")
print(f"URL: {response.get_request_url()}")

### ✅ Method 1: `get_dict()` (Always worked)

Returns the raw API response with nested JSON structure.

In [ ]:
# get_dict() returns raw nested JSON
raw_dict = response.get_dict()

print("Raw response structure:")
print(f"  Top-level keys: {list(raw_dict.keys())}")
print(f"  Total size: {len(json.dumps(raw_dict))} characters")
print("\nSample of nested structure:")
print(json.dumps(raw_dict['boxScoreTraditional']['homeTeam']['players'][0], indent=2)[:500] + "...")

### 🎉 Method 2: `get_normalized_json()` (NOW FIXED!)

**Before fix**: Returned `{}`  
**After fix**: Returns normalized, flat data structure

In [ ]:
# get_normalized_json() now returns properly normalized data!
normalized_json = response.get_normalized_json()
normalized_data = json.loads(normalized_json)

print("🎉 Normalized response (FIXED!):\n")
print(f"  Data sets available: {list(normalized_data.keys())}")
print()

for dataset_name, records in normalized_data.items():
    print(f"  📋 {dataset_name}: {len(records)} records")
    if records:
        print(f"     Fields: {list(records[0].keys())}")
        print()

print(f"\nTotal size: {len(normalized_json)} characters (was 2 before fix!)")

### 📊 Method 3: `get_normalized_dict()` (Also fixed!)

Same as `get_normalized_json()` but returns a Python dict instead of JSON string.

In [ ]:
# get_normalized_dict() also works now
normalized_dict = response.get_normalized_dict()

print("Normalized dict structure:")
print(f"  Type: {type(normalized_dict)}")
print(f"  Keys: {list(normalized_dict.keys())}")
print(f"  Same as JSON output: {normalized_dict == normalized_data}")